# Imports

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import re
import sys
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import torch

from assembly import Assembly
from utils import load_yaml, num_params

In [4]:
sns.set(font_scale=1.25, style="whitegrid")

# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print(f"Using device = {device}")

NUM_CORES = os.cpu_count()
if hasattr(os, "sched_getaffinity"):
    # This function is only available on certain platforms. When running with Slurm, it can tell us the true
    # number of cores we have access to.
    NUM_CORES = len(os.sched_getaffinity(0))
print (f"Using {NUM_CORES} cores.")

Using 8 cores.


# Counting Compute

Figuring out how to count params and FLOPs, or whatever might be a good proxy for amount of compute.

In [20]:
from fvcore.nn import FlopCountAnalysis
from torchtnt.utils.flops import FlopTensorDispatchMode

def print_sizes(cfile):
    cfg = load_yaml(cfile)
    assembly = cfg.get("assembly", cfg.get("stages", cfg.get("src_stages")))
    model = Assembly(assembly, cfg.get("head"), input_shape=[3, 224, 224])
    model.train()
    print(f"Number of parameters: {num_params(model):.3e}")
    for i, p in enumerate(model.parts):
        print(f"Num params in part {i+1} ({p.__class__.__name__}): {num_params(p):.3e}")

    dummy_img = torch.randn(1, 3, 224, 224)
    flops = FlopCountAnalysis(model, dummy_img)
    print(f"\nfvcore FLOPs: {flops.total()}")
    # print(f"fvcore FLOPs by module: {flops.by_module()}")
    # print(f"fvcore FLOPs by operator: {flops.by_operator()}")
    with FlopTensorDispatchMode(model) as ftdm:
        _ = model(dummy_img)
        print(f"\nTorchTNT FLOPs: {sum(ftdm.flop_counts[''].values())}")
        print(f"\nTorchTNT FLOPs breakdown:")
        for k, v in ftdm.flop_counts.items():
            print(f"    {k}: {v}")

In [21]:
print_sizes(Path("../across-scales/mobilenet-v3.yml"))

/Users/ntraft/miniconda3/envs/stitch/lib/python3.11/site-packages/torch/fx/graph.py:1062: UserWarning: erase_node(x) on an already erased node
  warnings.warn(f"erase_node({to_erase}) on an already erased node")
/Users/ntraft/miniconda3/envs/stitch/lib/python3.11/site-packages/torch/fx/graph.py:1062: UserWarning: erase_node(x) on an already erased node
  warnings.warn(f"erase_node({to_erase}) on an already erased node")
/Users/ntraft/miniconda3/envs/stitch/lib/python3.11/site-packages/torch/fx/graph.py:1062: UserWarning: erase_node(x) on an already erased node
  warnings.warn(f"erase_node({to_erase}) on an already erased node")
/Users/ntraft/miniconda3/envs/stitch/lib/python3.11/site-packages/torch/fx/graph.py:1062: UserWarning: erase_node(x) on an already erased node
  warnings.warn(f"erase_node({to_erase}) on an already erased node")
/Users/ntraft/miniconda3/envs/stitch/lib/python3.11/site-packages/torch/fx/graph.py:1062: UserWarning: erase_node(x) on an already erased node
  warning

Number of parameters: 5.483e+06
Num params in part 1 (Subnet): 4.368e+03
Num params in part 2 (Subnet): 4.440e+03
Num params in part 3 (Subnet): 1.033e+04
Num params in part 4 (Subnet): 4.198e+04
Num params in part 5 (Subnet): 3.208e+04
Num params in part 6 (Subnet): 6.993e+05
Num params in part 7 (Subnet): 4.292e+05
Num params in part 8 (Subnet): 4.261e+06

fvcore FLOPs: 238635840

TorchTNT FLOPs: 216589760

TorchTNT FLOPs breakdown:
    : defaultdict(<class 'int'>, {'convolution.default': 215309760, 'addmm.default': 1280000})
    parts.0: defaultdict(<class 'int'>, {'convolution.default': 29904896})
    parts.0.net: defaultdict(<class 'int'>, {'convolution.default': 29904896})
    parts.0.net.conv_stem: defaultdict(<class 'int'>, {'convolution.default': 5419008})
    parts.0.net.blocks.0.0.conv_dw: defaultdict(<class 'int'>, {'convolution.default': 1806336})
    parts.0.net.blocks.0.0.conv_pw: defaultdict(<class 'int'>, {'convolution.default': 3211264})
    parts.0.net.blocks.1.0.con

In [22]:
print_sizes(Path("../across-scales/resnet-50.yml"))

/Users/ntraft/miniconda3/envs/stitch/lib/python3.11/site-packages/torch/fx/graph.py:1062: UserWarning: erase_node(x) on an already erased node
  warnings.warn(f"erase_node({to_erase}) on an already erased node")


Number of parameters: 2.556e+07
Num params in part 1 (Subnet): 8.454e+04
Num params in part 2 (Subnet): 1.408e+05
Num params in part 3 (Subnet): 3.794e+05
Num params in part 4 (Subnet): 8.402e+05
Num params in part 5 (Subnet): 1.512e+06
Num params in part 6 (Subnet): 5.586e+06
Num params in part 7 (Subnet): 6.040e+06
Num params in part 8 (Subnet): 1.097e+07

fvcore FLOPs: 4144854528

TorchTNT FLOPs: 4089184256

TorchTNT FLOPs breakdown:
    : defaultdict(<class 'int'>, {'convolution.default': 4087136256, 'addmm.default': 2048000})
    parts.0: defaultdict(<class 'int'>, {'convolution.default': 349224960})
    parts.0.net: defaultdict(<class 'int'>, {'convolution.default': 349224960})
    parts.0.net.conv1: defaultdict(<class 'int'>, {'convolution.default': 118013952})
    parts.0.net.layer1.0.conv1: defaultdict(<class 'int'>, {'convolution.default': 12845056})
    parts.0.net.layer1.0.conv2: defaultdict(<class 'int'>, {'convolution.default': 115605504})
    parts.0.net.layer1.0.conv3: 

In [23]:
print_sizes(Path("../across-scales/swin-t.yml"))

/Users/ntraft/miniconda3/envs/stitch/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/ntraft/miniconda3/envs/stitch/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Swin_T_Weights.IMAGENET1K_V1`. You can also use `weights=Swin_T_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/Users/ntraft/miniconda3/envs/stitch/lib/python3.11/site-packages/torch/fx/graph.py:1062: UserWarning: erase_node(x) on an already erased node
  warnings.warn(f"erase_node({to_erase}) on an already erased node")


Number of parameters: 2.829e+07
Num params in part 1 (Subnet): 4.896e+03
Num params in part 2 (Subnet): 2.247e+05
Num params in part 3 (Subnet): 7.450e+04
Num params in part 4 (Subnet): 8.918e+05
Num params in part 5 (Subnet): 2.964e+05
Num params in part 6 (Subnet): 1.066e+07
Num params in part 7 (Subnet): 1.183e+06
Num params in part 8 (Subnet): 1.495e+07


parts.1.net.features.1.0.stochastic_depth



fvcore FLOPs: 4509232128

TorchTNT FLOPs: 4490566656

TorchTNT FLOPs breakdown:
    : defaultdict(<class 'int'>, {'convolution.default': 14450688, 'addmm.default': 4162566144, 'bmm.default': 140141568, 'mm.default': 173408256})
    parts.0: defaultdict(<class 'int'>, {'convolution.default': 14450688})
    parts.0.net: defaultdict(<class 'int'>, {'convolution.default': 14450688})
    parts.0.net.features.0.0: defaultdict(<class 'int'>, {'convolution.default': 14450688})
    parts.1: defaultdict(<class 'int'>, {'addmm.default': 693633024, 'bmm.default': 59006976})
    parts.1.net: defaultdict(<class 'int'>, {'addmm.default': 693633024, 'bmm.default': 59006976})
    parts.1.net.features.1.0.mlp: defaultdict(<class 'int'>, {'addmm.default': 231211008})
    parts.1.net.features.1.0.mlp.0: defaultdict(<class 'int'>, {'addmm.default': 115605504})
    parts.1.net.features.1.0.mlp.3: defaultdict(<class 'int'>, {'addmm.default': 115605504})
    parts.1.net.features.1.1.mlp: defaultdict(<class '